# Data Collection

## About

### Summary

Business of Apps allowed this project to be possible by providing a dataset of ...

### Web Scraping Goals (revisar)

0. Configure web scraping system;	
1. Reach target pages;
2. Locate the charts;
3. Identify each chart's title;
4. Construct iframe selectors using the chart titles;
5. Save the iframe selectors in a list;
6. Use the iframe list to perform actions within them;
7. Hover over each chart column with the mouse pointer;
8. Save the values that appear in the page source when hovering over each chart column.

### Raw Data Preparation

### Data Version Control

Para garantir a reprodutibilidade dessa etapa do experimento, desenvolvi um sistema simples de controle de versão de dados (DVC) Ad-Hoc. 

Esse sistema permitiu a comparação automática dos dados coletados em execuções futuras com os dados obtidos anteriormente, ou seja, com os dados que já haviam sido armazenados no estudo.
Apos os dados terem sido coletados, eles foram registrados como a primeira versão dos dados, então salvos dentro do repositório e no bloco de codigo em uma estrutura de dados apropriada. 

Novos dados coletados são comparados com essa versão dos dados, quando o jupyter notebook é executado novamente, e se houver diferenças um aviso é lançado no output e pipelinee os novos dados são salvos como a nova versão dos dados automaticamente. Em caso de serem identicos, então o sistema não faz nada e o pipeline é interrompido.

### Reproducibility Note



## Web Scraping

### Setup

In [1]:
from playwright.async_api import async_playwright
from IPython.display import Image, display
from tqdm.asyncio import tqdm
import pandas as pd
pd.set_option('display.max_rows', None)

targets_url = ['https://www.businessofapps.com/data/instagram-statistics/', 
               'https://www.businessofapps.com/data/tik-tok-statistics/']
target_data_selector = 'infogram-embed'
target_data_iframe_selector = 'iframe[title="{chart_iframe_title_goes_here}"]'

p = await async_playwright().start()
browser = await p.chromium.launch()
context = await browser.new_context()

The web scraping system was successfully configured. The results were:

- All web scraping parameters were set up;
- Playwright API was successfully initialized.

Web scraping goals reached:

1. ~~Configure web scraping stack;~~	
2. Reach target pages;
3. Locate the charts;
4. Identify each chart's title;
5. Construct iframe selectors using the chart titles;
6. Save the iframe selectors in a list;
7. Use the iframe list to perform actions within them;
8. Hover over each chart column with the mouse pointer;
9. Scrape the values that appear in the page source when hovering over each chart column.

### Target Pages Reacher

In [ ]:
ig = await context.new_page()
tk = await context.new_page()

await ig.goto(targets_url[0])
print(await ig.title())
display(Image(await ig.screenshot()))

await tk.goto(targets_url[1])
print(await tk.title())
display(Image(await tk.screenshot()))

The web scraping system was successfully initiated. The results were:
- Target URLs was successfully reached.

Web scraping goals reached:

1. ~~Configure web scraping stack;~~	
2. ~~Reach target pages;~~
3. Locate the charts;
4. Identify each chart's title;
5. Construct iframe selectors using the chart titles;
6. Save the iframe selectors in a list;
7. Use the iframe list to perform actions within them;
8. Hover over each chart column with the mouse pointer;
9. Scrape the values that appear in the page source when hovering over each chart column.

### Target Data (charts) Locator

In [ ]:
async def chart_finder(page: 'PageObject', chart_selector: str, screenshot: bool = True) -> list:
    """
    Encontra gráficos em uma página e retorna os seletores de iframe correspondentes.

    Args:
        page (PageObject): Objeto da página do Playwright.
        chart_selector (str): Seletor CSS para identificar os gráficos na página.
        screenshot (bool, optional): Se True, tira uma captura de tela de cada gráfico encontrado. Padrão é True.

    Returns:
        list: Lista de seletores de iframe correspondentes aos gráficos encontrados.
    """
    founded_iframes_selectors = []
    founded_charts = await page.query_selector_all(f'.{chart_selector}')

    for target in founded_charts:
        await target.scroll_into_view_if_needed()
        await page.wait_for_timeout(2000)

        title = await target.get_attribute('data-title')
        iframe_selector = f'iframe[title="{title}"]'
        founded_iframes_selectors.append(iframe_selector)

        if screenshot:
            page_title = await page.title()
            print(f'Page: {page_title} || Chart iframe: {iframe_selector}')
            display(Image(await page.screenshot()))

    return founded_iframes_selectors

ig_iframes = await chart_finder(page=ig, chart_selector=target_data_selector)
tk_iframes = await chart_finder(page=tk, chart_selector=target_data_selector)

### Target Data Iframe Selector Constructor

In [ ]:
for iframe in [ig_iframes, tk_iframes]:
    print(iframe)

In [ ]:
iframe_to_remove_1 = 'iframe[title="Social App Users"]'
iframe_to_remove_2 = 'iframe[title="TikTok Quarterly Downloads"]'

def remove_iframe(iframes):
    if iframe_to_remove_1 in iframes:
        iframes.remove(iframe_to_remove_1)
    if iframe_to_remove_2 in iframes:
        iframes.remove(iframe_to_remove_2)
    return iframes

ig_iframes = remove_iframe(ig_iframes)
tk_iframes = remove_iframe(tk_iframes)

In [ ]:
for iframe in [ig_iframes, tk_iframes]:
    print(iframe)

The web scraping system located all target data (charts). The results were:
- Target data iframes allocated in a list.

Web scraping goals reached:

1. ~~Configure web scraping stack;~~	
2. ~~Reach target pages;~~
3. ~~Locate the charts;~~
4. ~~Identify each chart's title;~~
5. ~~Construct iframe selectors using the chart titles;~~
6. ~~Save the iframe selectors in a list;~~
7. Use the iframe list to perform actions within them;
8. Hover over each chart column with the mouse pointer;
9. Scrape the values that appear in the page source when hovering over each chart column.

### Target Data Webscraper

In [ ]:
async def barchart_scraper(page: 'PageObject', target_iframes: list) -> pd.DataFrame:
    """
    target_iframes: list within iframes target selectors
    """
    scraped_barchart_data = []

    print('Webscraping process started')
    print(f'Page: {await page.title()}')
    print()
    for barchart in target_iframes:
        barchart_iframe_title = f'{await page.frame_locator(barchart).locator("title").inner_text()}'

        columns_qtd = await page.frame_locator(barchart).locator('.igc-graph .igc-column').all()
        columns_qtd = len(columns_qtd) + 1

        # Mensagem inicial sem a barra de progresso
        print(f'-> barchart ({barchart_iframe_title}):', end='')

        # Inicialize uma barra de progresso para o gráfico atual
        pbar = tqdm(total=columns_qtd - 1, desc='Progress', unit=' Datapoint', leave=True)
        
        for i in range(1, columns_qtd):
            await page.frame_locator(barchart).locator(f'path:nth-child({i})').hover()

            target_time_series_category = await page.frame_locator(barchart).locator('.tt_text').inner_text()
            target_time_series_value = await page.frame_locator(barchart).locator('.tt_value').inner_text()

            # Adicione os dados extraídos à lista
            scraped_barchart_data.append({
                'iframe_title': barchart_iframe_title,
                'timeseries_category': target_time_series_category,
                'timeseries_value': target_time_series_value
            })

            # Atualize a barra de progresso
            pbar.update(1)

        # Finalize a barra de progresso
        pbar.close()

        # Imprima a mensagem de conclusão após fechar a barra de progresso
        print(f'-> barchart ({barchart_iframe_title}) webscraping ended')
        print()
    
    print('Webscraping process ended')
    print('-'*70)
    print()

    return pd.DataFrame(scraped_barchart_data)

In [ ]:
raw_scraped_data = []
raw_scraped_data.append(await barchart_scraper(page=ig, target_iframes=ig_iframes))
raw_scraped_data.append(await barchart_scraper(page=tk, target_iframes=tk_iframes))

## Raw Scraped Data Preparation

### quarter_label

In [ ]:
# concat all dataframes
prep_data = pd.concat(raw_scraped_data, ignore_index=True)
prep_data.head(1)

In [ ]:
scraped_data = pd.DataFrame(columns=['quarter', 'quarter_label', 'ig_maus', 'ig_revs', 'tk_maus', 'tk_revs'])
scraped_data 

In [ ]:
scraped_data['quarter_label'] = prep_data['timeseries_category'].unique()

In [ ]:
scraped_data.head()

### quarter

In [ ]:
# Função para converter uma string de trimestre para a data de fechamento trimestral 
def converter_para_data(trimestre):
    tri, ano = trimestre.split()
    tri = int(tri[1])
    if tri == 1:
        mes, dia = 3, 31
    elif tri == 2:
        mes, dia = 6, 30
    elif tri == 3:
        mes, dia = 9, 30
    elif tri == 4:
        mes, dia = 12, 31
    return pd.Timestamp(year=int(ano), month=mes, day=dia)

# Aplicar a transformação ao DataFrame df
scraped_data['quarter'] = scraped_data['quarter_label'].apply(converter_para_data)
scraped_data.head()

In [ ]:
scraped_data = scraped_data.sort_values(by='quarter').reset_index(drop=True)
scraped_data.head()

In [ ]:
# Definir o ano atual
current_year = pd.Timestamp.now().year

# Filtrar os dados para manter apenas os últimos 5 anos
scraped_data = scraped_data[scraped_data['quarter'].dt.year >= (current_year - 6)]

# Resetar o índice
scraped_data.reset_index(drop=True, inplace=True)

# Exibir os dados filtrados
scraped_data

### maus and revenues

In [ ]:
# Função para preencher o scraped_data com os valores do prep_data
def preencher_scraped_data(prep_data, scraped_data):
    for index, row in prep_data.iterrows():
        if 'Instagram' in row['iframe_title']:
            if 'monthly app users' in row['iframe_title']:
                scraped_data.loc[scraped_data['quarter_label'] == row['timeseries_category'], 'ig_maus'] = row['timeseries_value']
            elif 'revenues' in row['iframe_title']:
                scraped_data.loc[scraped_data['quarter_label'] == row['timeseries_category'], 'ig_revs'] = row['timeseries_value']
        elif 'TikTok' in row['iframe_title']:
            if 'MAUs' in row['iframe_title']:
                scraped_data.loc[scraped_data['quarter_label'] == row['timeseries_category'], 'tk_maus'] = row['timeseries_value']
            elif 'revenues' in row['iframe_title']:
                scraped_data.loc[scraped_data['quarter_label'] == row['timeseries_category'], 'tk_revs'] = row['timeseries_value']
    return scraped_data

# Preencher o scraped_data
scraped_data = preencher_scraped_data(prep_data, scraped_data)
scraped_data = scraped_data.astype({'ig_maus': 'int64', 'ig_revs': 'int64', 'tk_maus': 'int64', 'tk_revs': 'int64'})

# Exibir o resultado
display(scraped_data.head())
display(scraped_data.tail())
display(scraped_data.dtypes)

## Scraped Data Exporation

### Setup

In [ ]:
import os 
import hashlib
import pandas as pd

os.chdir('..')

In [ ]:
def generate_data_sha256_hash(data: pd.DataFrame) -> None:
    """
    Gera uma hash SHA256 para um DataFrame.
    """
    hash_object = hashlib.sha256()
    hash_object.update(data.to_string().encode())
    hash = hash_object.hexdigest()
    print('Hash do DataFrame carregado:', hash)

def data_integrity_test(data: pd.DataFrame) -> None:
    """
    Para garantir a reprodutibilidade deste notebook, o hash SHA-256 do DataFrame em estado raw (sem transformações) deve ser exatamente:
    50f08805a0b891323bc75e43010e3622e80a2607c8cf585729b153f7703adf6f

    Esta função garante a integridade dos dados. Se os dados estiverem íntegros, um output de conformidade será exibido.
    Caso contrário, o notebook lançará um erro, indicando que o experimento original não é reprodutível.
    """
    ex_hash = '50f08805a0b891323bc75e43010e3622e80a2607c8cf585729b153f7703adf6f'
    hash_object = hashlib.sha256()
    hash_object.update(data.to_string().encode())
    hash = hash_object.hexdigest()

    print('Hash do DataFrame esperado: ',ex_hash)

    if hash == ex_hash:
        print('-> DataFrame carregado passou no teste de intrigade. Hashes constaram como idênticas.')
        print('-> Experimento/análise original pode ser reproduzido.')

    else:
        raise ValueError('DataFrame carregado NÃO passou no teste de intrigade. O experimento/análise original NÃO pode ser reproduzido.')

### Data Export

In [ ]:
scraped_data

In [ ]:
# Converter as colunas numéricas para int
scraped_data[['ig_maus', 'ig_revs', 'tk_maus', 'tk_revs']] = scraped_data[['ig_maus', 'ig_revs', 'tk_maus', 'tk_revs']].astype(int)

In [ ]:
scraped_data.dtypes

In [ ]:
# remove a ultima linha do df
scraped_data = scraped_data[:-1]
scraped_data.to_parquet('data/raw/scraped_data.parquet')